# Explore NHANES

This is a first swing at exploring NHANES datasets. Currently just working with the 2021-2023 survey and lumping both days into a single dataset.

An issue that needs to be fixed here is that the grams of PBPs consumed below are based on the total foods/recipes, but are not broken down into what proportion are actually PBPs. Need to add the recipe doc that translates recipes into constituent ingredients and proportions to get proper PBP consumption.

## Set Working Directory

When you open a notebook, the default working directory will be the folder that notebook is in. We want it to be the top (root) directory of the project, `ds1_nhanes`.

First, we need to mount our Google Drive, which contains the `ds1_nhanes` folder. The following chunk will mount the drive (if in Google Colab) and set the working directory to the root of the project folder.

In [ ]:
import os

try:
  from google.colab import drive
  drive.mount('/content/drive')
  os.chdir('/content/drive/MyDrive/ds1_nhanes/')
except:
  from pathlib import Path
  os.chdir(Path(os.getcwd()).parent)

print(os.getcwd())

Bingo bongo, we're good to go.

## Load Data and Libraries

In [ ]:
import pandas as pd
import numpy as np
import re

The DR1IFF_L and DR2IFF_L datasets include daily food intake over two days for respondents identified by the SEQN or respondent sequence identifier. The surveys were conducted in waves and we aim to combine two waves of data survey responses from 2017-2020 and 2021-2023. Given the size of these datasets we tried to pull only the columns that are relevant for our analysis are read for both days and both years, however, usecols does not work on .XPT files.

In [ ]:
# Read in the four datasets with specified columns
dr1_17 = pd.read_sas('data/raw/nhanes_2017_2020/P_DR1IFF.xpt')
dr2_17 = pd.read_sas('data/raw/nhanes_2017_2020/P_DR2IFF.xpt')
dr1_21 = pd.read_sas('data/raw/nhanes_2021_2023/DR1IFF_L.xpt')
dr2_21 = pd.read_sas('data/raw/nhanes_2021_2023/DR2IFF_L.xpt')

# Create a list of all the datasets
datasets = [dr1_21, dr2_21, dr1_17, dr2_17]

# Define the relevant columns for this analysis
relevant_cols = ['SEQN', # response_sequence, unique identifier for each respondent
                 'WTDR2D', # weight_day_2_dietary, the weighting factor given to
                 # the second day depending on how many days the respondents reported
                 'WTDR2DPP', # different name in 2017 - 2020 data
                 'DR1IGRMS', # grams, total grams of food consumed, labeled DR1GRMS or DR2GRMS
                 # based on which day it was reported
                 'DR2IGRMS', # labeled DR1GRMS or DR2GRMS based on which day it was reported
                 'DR1IFDCD', # usda_food_code, food identifier
                 'DR2IFDCD'] # labeled DR1GRMS or DR2GRMS based on which day it was reported
for dataset in datasets:
  dataset.drop(columns=[col for col in dataset.columns if col not in relevant_cols], inplace=True)

#Look up what columns are relevant and load only what is relevant, check class notes - LB
print(dr1_21.info())
print(dr2_21.info())
print(dr1_17.info())
print(dr2_17.info())
dr1_21.head()


## Explore Dietary Recall Data

Compare the dimensions of the df with the number of unique SEQN numbers (respondent ids)

In [ ]:
# Compare rows to unique respondent IDs
# first get number of rows
rows = dr1_21.shape[0]
unique_seqns = dr1_21['SEQN'].nunique()
print(f"{rows} rows and {unique_seqns} unique SEQN numbers in DR1.")

There are far more rows than unique respondents. This is because for each respondent, there is one row for each individual food they consumed.

Check out how many unique food codes there are:

In [ ]:
n_codes = dr1_21['DR1IFDCD'].nunique()
print(f"There are {n_codes} unique food codes")

In [ ]:
# Check SEQNs between dr1 and dr2

diff = set(dr1_17['SEQN']).difference(dr2_17['SEQN'])
print(f"{len(diff)} SEQNs missing from DR2 that were in DR1 in the 2017 - 2020 wave")
diff2 = set(dr2_17['SEQN']).difference(dr1_17['SEQN'])
print(f"{len(diff2)} SEQNs missing from DR1 that were in DR2 in the 2017 - 2020 wave")

diff = set(dr1_21['SEQN']).difference(dr2_21['SEQN'])
print(f"{len(diff)} SEQNs missing from DR2 that were in DR1 in the 2021 - 2023 wave")
diff2 = set(dr2_21['SEQN']).difference(dr1_21['SEQN'])
print(f"{len(diff2)} SEQNs missing from DR1 that were in DR2 in the 2021 - 2023 wave")



For the 2017 - 2020 wave, there are 1804 respondants for day 1 that did not report in day 2, and two respondent who reported in day 2 but not day 1.

For the 2021 - 2023 wave, there are 873 respondants for day 1 that did not report in day 2, and one respondent who reported in day 2 but not day 1.  people when we join.

The two-day weights (WTDR2) was adjusted based on the day 1 weights (WTDR1) and further adjusting for additional non-response for the second recall, so we will drop the respondents that only respond in both days.


## Join with FPED

Join FPED to each of the four DR datasets. We have to do this and aggregate within each DR before we combine the DRs together, otherwise we get cartesian merges.

In [ ]:
fped = pd.read_csv('data/miscellany/FPED_1720.csv')
fped.columns = fped.columns.str.lower()
fped.columns = fped.columns.str.replace(" ", "_")

fped.info()

Rename columns in DR datasets that are not consistent (FDCD, GRMS, and 2 day weights). This will make it easier map over the list of all datasets when we aggregate

In [ ]:
# Function to rename columns across all datasets
# These are the ones we care about that differ between DR dataset
def rename_columns(df):
    new_columns = {}
    for col in df.columns:
        if re.search(r'FDCD$', col):
            new_columns[col] = 'food_code'
        elif re.search(r'GRMS$', col):
            new_columns[col] = 'grams'
        elif re.search(r'^WTDR2', col):
            new_columns[col] = 'weight_2d'
        else:
            new_columns[col] = col

    df = df.rename(columns=new_columns)
    return df

# Rename columns in each dataset
datasets_renamed = list(map(rename_columns, datasets))

print("\nRenamed Datasets:\n")
list(map(lambda df: df.info(), datasets_renamed))


Merge each DR with FPED

In [ ]:
# Map over our list of datasets and merge each out with FPED
def merge_with_fped(df):
    return df.merge(fped, left_on='food_code', right_on='foodcode', how='left')

dfs_fped = list(map(merge_with_fped, datasets_renamed))

print("\nDatasets with FPED:\n")
list(map(lambda df: df.info(), dfs_fped))

Now we can concat DR1 and DR2 together for each wave

In [ ]:
# Combine first and second DF to make a df for 2021-2023
df_21 = pd.concat([dfs_fped[0], dfs_fped[1]], ignore_index=True)
df_21.info()

# Combine second and third DF to make df for 2017-2020
df_17 = pd.concat([dfs_fped[2], dfs_fped[3]], ignore_index=True)
df_17.info()

Get food group totals for each food code for each person. Take grams, divide by 100, then multiply by every food group category.

In [ ]:
# Put them back into a list
waves = [df_17, df_21]

def get_food_group_totals(df):
    df.loc[:, 'f_total_(cup_eq)':'a_drinks_(no._of_drinks)'].multiply(df['grams']/100, axis=0)
    return df

waves_fped = list(map(
    get_food_group_totals,
    waves
))

waves_fped[0].head()

Group by SEQN and aggregate FPED variables

In [ ]:
## Grouping by SEQN within each DR dataset
# Set aggregation functions so we don't have to do them all manually
# Everything except food code and description in FPED should be summedf
cols_to_sum = fped.columns[2:]

# Set aggregation functions
aggs = {col: 'sum' for col in cols_to_sum}
aggs['SEQN'] = 'first'
aggs['weight_2d'] = 'unique'
aggs['grams'] = 'sum'

# Aggregate each dataset
waves_grouped = list(map(
    lambda df:
        df.groupby('SEQN').agg(aggs),
    waves_fped
))

waves_grouped[0].head()

Now we are down to 1 row per SEQN per wave, including both days of dietary recall.

Finally we can concat both waves into a single df and divide 2day weights by 2


In [ ]:
# Combine waves
df = pd.concat(waves_grouped, ignore_index=True)

# Divide 2day weights by 2 since we are combining two waves
df['weight_2d'] = df['weight_2d'] / 2

# Rearrange columns by bringing important ones to front
# Leaving in grams just as a check
front_cols = ['SEQN', 'weight_2d', 'grams']
cols = front_cols + [col for col in df.columns if col not in front_cols]
df = df[cols]

df.info()

In [ ]:
# Make the SEQN column an integer and weight_2d a float with two decimal places

df['SEQN'] = df['SEQN'].astype(int)

df['weight_2d'] = df['weight_2d'].astype(float)
df['weight_2d'] = df['weight_2d'].round(2)

df

This is hopefully a nice clean DF with one row per SEQN, both DR days, and two waves lumped together.

## Add Demographics

The NHANES dataset includes demographics for each respondent. The following steps merge key demographics with each respondent id number SEQN.

Load demographic data as xpt:

In [ ]:
demos_17 = pd.read_sas('data/raw/nhanes_2017_2020/P_DEMO.xpt')
demos_21 = pd.read_sas('data/raw/nhanes_2021_2023/DEMO_L.xpt')

demos_17.info()
demos_21.info()

# combine into a single dataset
demos = pd.concat([demos_17, demos_21], ignore_index=True)
demos.info()

Take the columns SEQN, age, gender, race, education, and ratio of family income to poverty:

In [ ]:
demos = demos[['SEQN', 'RIAGENDR', 'RIDAGEYR', 'RIDRETH3', 'DMDEDUC2', 'INDFMPIR']]
# rename the demo columns
demos.columns = ['SEQN', 'gender', 'age', 'race', 'education', 'income_ratio']
demos.info()

Merge demographics with our dietary intake data:

In [ ]:
# check to see

diff = set(df['SEQN']).difference(demos['SEQN'])
print(f"{len(diff)} SEQNs missing from demographics that were in dietary intake")
diff2 = set(demos['SEQN']).difference(df['SEQN'])
print(f"{len(diff2)} SEQNs missing from dietary intake that were in demographics")

# Now join our demos with dietary intake data
df = df.merge(demos, on='SEQN', how='left')
df.info()

Recode demographic variables. Coding schemes are available at the NHANES website in the documentation beside each dataset. We are splitting the income to poverty ratio into quartiles.

In [ ]:
# Gender
df['gender'] = df['gender'].apply(lambda x: ('Female' if x == 2 else 'Male'))

# Education
df['education'] = df['education'].apply(
  lambda x: (
    'Less than 9th grade' if x == 1
    else '9th to 11th grade' if x == 2
    else 'High school/GED' if x == 3
    else 'Some college or AA' if x == 4
    else 'College graduate or above' if x == 5
    else "Don\'t know"
  )
)

# Race
df['race'] = df['race'].apply(
  lambda x: (
    'Mexican American' if x == 1
    else 'Other Hispanic' if x == 2
    else 'White' if x == 3
    else 'Black' if x == 4
    else 'Asian' if x == 5
    else 'Other or Multi'
  )
)

# Income to poverty ratio
df['income_ratio_qs'] = pd.qcut(
  x = df['income_ratio'],
  q = 4,
  duplicates='drop'
)
df.head()

## Identify plant-based protein eaters

Get the column names which identify food codes as containing any amount of
legumes, nuts/seeds, or soy. Then identify the food codes that are PBPs and label as has PBPs

In [ ]:
# Keywords we will use to find column names
keywords = ['pf_legumes', 'pf_nutsds', 'pf_soy']
pbp_columns = [col for col in df.columns if any(keyword in col for keyword in keywords)]
print(pbp_columns)

# Condition: Any of the selected columns has a value greater than 1
df['has_pbp'] = (df[pbp_columns] > 1).any(axis=1)
print(df.head())

# Check that this worked
df[df['has_pbp'] == True][pbp_columns + ['has_pbp']].head(10)

# Sum the total grams of pbp in a new column
df['grams_from_pbp'] = df[pbp_columns].sum(axis=1)
print(df.head())



In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19386 entries, 0 to 19385
Data columns (total 48 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   SEQN                       19386 non-null  int64   
 1   weight_2d                  19386 non-null  float64 
 2   grams                      19386 non-null  float64 
 3   f_total_(cup_eq)           19386 non-null  float64 
 4   f_citmlb_(cup_eq)          19386 non-null  float64 
 5   f_other_(cup_eq)           19386 non-null  float64 
 6   f_juice_(cup_eq)           19386 non-null  float64 
 7   v_total_(cup_eq)           19386 non-null  float64 
 8   v_drkgr_(cup_eq)           19386 non-null  float64 
 9   v_redor_total_(cup_eq)     19386 non-null  float64 
 10  v_redor_tomato_(cup_eq)    19386 non-null  float64 
 11  v_redor_other_(cup_eq)     19386 non-null  float64 
 12  v_starchy_total_(cup_eq)   19386 non-null  float64 
 13  v_starchy_potato_(cup_eq)  1938

Let's also save this as a csv so we can play around with it elsewhere:

In [ ]:
df.to_csv('data/clean/df_nhanes_2017_2023.csv')

## Graphs

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

Set a common theme for our plots:

In [ ]:
sns.set_theme(
    style="ticks",
    rc= {
      "axes.spines.right": False,
      "axes.spines.top": False,
      "figure.figsize": (6, 6)
    }
  )

PBP consumption by gender. Note that we are using the 2-day weights

In [ ]:
# pbp consumption by gender
ax = sns.barplot(
  data=df,
  y='grams_from_pbp',
  x='education',
  hue='gender',
  order=[
    "Don't know",
    'Less than 9th grade',
    'High school/GED',
    'Some college or AA',
    'College graduate or above'
  ],
  weights='weight_2d',
  errorbar=('ci', 95)
)
ax.set(
  ylabel = 'Proportion of grams from PBP',
  xlabel = 'Education',
  title = 'PBP Consumption by Education and Gender'
)
plt.xticks(rotation=45)

# Save plot
plt.tight_layout()
plt.savefig('outputs/checkin_1/pbp_consumption_by_education.png')

plt.show()
# Would like to adjust names horizontally to line up better after rotation, hjust arg?

In [ ]:
# pbp consumption by race
ax = sns.barplot(
  data=df,
  y='grams_from_pbp',
  x='race',
  hue='gender',
  weights='weight_2d',
  errorbar=('ci', 95)
)
ax.set(
  xlabel = 'Race',
  ylabel = 'Percentage of grams from PBP',
  title = 'PBP as Proportion of Consumption by Race and Gender'
)
plt.xticks(rotation=45)
plt.tight_layout()

# Save it
plt.savefig('outputs/checkin_1/pbp_consumption_by_race.png')

# Show it
plt.show()

In [ ]:
# pbp consumption by poverty ratio
ax = sns.barplot(
  data = df,
  y='grams_from_pbp',
  x='income_ratio_qs',
  hue='gender',
  weights='weight_2d',
  errorbar=('ci', 95)
)
ax.set(
  xlabel = 'Quartiles of Income to Poverty Ratio',
  ylabel = 'Percentage of grams from PBP',
  title = 'PBP as Proportion of Consumption by Income and Gender'
)
plt.xticks(rotation=45)
plt.tight_layout()

# Save it
plt.savefig('outputs/checkin_1/pbp_consumption_by_income.png')

# Show it
plt.show()

## Test a Table

Just figuring out how to make a LaTeX table

In [ ]:
# Make a smaller DF to play around with
small_df = df[['weight_2d', 'grams', 'gender', 'grams_from_pbp']].head()
print(small_df)

In [ ]:
# Rename columns to ditch underscores
small_df.columns = ['2 Day Weight', 'Total Grams Over 2 Days', 'Gender', 'Total PBP Grams']
print(small_df)

In [ ]:
small_df.to_latex(
  'outputs/checkin_1/test_table.tex',
  index=False,
  float_format="%.2f",
  label='test_table',
  caption='This is a test table',
  position='h'
)


Looks like this mostly works. Could use some proper formatting though